In [6]:
#imports
import pandas as pd
import time
import json
import requests

In [2]:
#load datasets, need to change filepath if using
df = pd.read_csv('C:\\Users\\C23Daniel.Joseph\\OneDrive - usafa.edu\\DS Capstone\\capstone\\AFBs', encoding = 'utf8')
df2 = pd.read_csv('C:\\Users\\C23Daniel.Joseph\\OneDrive - usafa.edu\\DS Capstone\\capstone\\AFB Raw Data', encoding = 'utf8')

In [3]:
#drops unnecessary columns
df2.drop(['Unnamed: 0','Data', 'place_id', 'licence', 'osm_type', 'osm_id', 'lat', 'lon',
          'boundingbox', 'address.town','address.county',  'address.ISO3166-2-lvl4',
          'address.country_code', 'address.amenity','address.house_number', 'address.man_made',
          'address.aeroway', 'address.hamlet', 'address.military', 'address.locality',
          'address.neighbourhood', 'address.village', 'address.suburb', 'address.historic',
          'address.township', 'address.building', 'address.residential', 'display_name',
          'address.municipality', 'address.quarter', 'address.office', 'State or area',
          'address.city_district', 'address.leisure', 'address.borough',
          'address.province', 'address.ISO3166-2-lvl6', 'address.tourism',
          'address.archipelago', 'address.state_district', 'address.road', 'address.city',
          'address.industrial', 'address.region', 'address.ISO3166-2-lvl3', 'address.postcode'],
         axis = 1,
         inplace = True
        )

#renames columns
df2.rename(columns = {'address.state':'State','address.country':'Country'}, inplace = True)

#drops bases that contain the mentioned words
df2 = df2[df2.Name.str.contains('Airport|Field|Airfield|Camp|Facility|Site')==False]

In [4]:
#initialize empty comlumn to eventually store neighboring zips
df2['Neighbor Zips'] = None

#initialize empty list to store neighboring zips by zip code
neighborListZips = []

In [7]:
#webscrape of geonames

#initialize variables for webscrape
country = 'US'
radius = '15'
user = 'twister101999'

neighborListZips = []

#webscrape
for row in df2.Zip:
    url = 'http://api.geonames.org/findNearbyPostalCodesJSON?postalcode=' + str(row) + '&country=' + country + '&radius=' + radius + '&username=' + user
    time.sleep(1)
    page = requests.get(url)
    
    neighborListZips.append(page.text)

In [9]:
#insert list into empty column
df2['Neighbor Zips'] = neighborListZips

#json-ify the column of neighbors
df2['Neighbor Zips'] = df2['Neighbor Zips'].apply(json.loads)

In [10]:
#function to extract zip codes from json

def extractZips(string):
    
    cellZips = []
    
    try:
        for zipCode in string['postalCodes']:
            cellZips.append(zipCode['postalCode'])
    except:
        pass
        
    return cellZips

In [11]:
#perform extractZips() on neighbor zips columns
df2['Neighbor Zips'] = df2['Neighbor Zips'].apply(extractZips)

In [12]:
df2

,Name,Location,Lat,Long,Zip,State,Country,Neighbor Zips
0,Altus Air Force Base,Altus,34.666390,-99.268060,None,Oklahoma,United States,[]
1,Joint Base Anacostia-Bolling,Southwest,38.842780,-77.016110,20332,District of Columbia,United States,[]
3,Arnold Air Force Base,Tullahoma,35.392500,-86.085830,37388,Tennessee,United States,"[37388, 37360, 37330]"
4,Barksdale Air Force Base,Bossier City,32.501940,-93.662780,71110,Louisiana,United States,"[71110, 71112, 71111, 71037, 71171]"
5,Beale Air Force Base,Marysville,39.136110,-121.436390,95903,California,United States,"[95903, 95692, 95977, 95681, 95901]"
...,...,...,...,...,...,...,...,...
233,Al Udeid Air Base,Qatar,25.117220,51.314720,None,الريان,قطر,[]
234,Ali Al Salem Air Base,Kuwait,29.346670,47.520560,None,الجهراء,الكويت,[]
235,Ahmad al-Jaber Air Base,Kuwait,28.934917,47.792028,None,الاحمدي,الكويت,[]
236,Muwaffaq Salti Air Base,Jordan,31.834170,36.787220,None,الزرقاء,الأردن,[]
